In [38]:
import pytraj as pt
import numpy as np
import pandas as pd
import tqdm
import sys
import gc
import copy
import nglview as nv

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [3]:
strucDir='structures/'
viewPDBname='Piezo_PIP2.prot_only.pdb'
trajName='Piezo_PIP2.prot_only.dcd'
topName='Piezo_PIP2.prot_only.parm7'
viewStruc=pt.load(strucDir+'Piezo_PIP2.prot_only.pdb')
iterTraj=pt.iterload(strucDir+trajName,
                     top=strucDir+topName)

print viewStruc
print iterTraj

pytraj.Trajectory, 1 frames: 
Size: 0.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           
pytraj.TrajectoryIterator, 1592 frames: 
Size: 2.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           


NGLView can allow us to look at the structure directly within the notebook.

Warning: google colab seems to have trouble with this as nglview apparently crashes the kernel

In [4]:
color_scheme=[
    ['blue','ARG LYS HIS HIP HID HIE HSD'],
    ['red','ASP GLU'],
    ['green','SER THR ASN GLN'],
    ['white','CYS SEC GLY PRO ALA VAL ILE LEU MET PHE TYR TRP']
]

In [68]:
view=nv.show_pytraj(viewStruc)
view.clear_representations()
view.add_representation('cartoon',color='white',alpha=.5)
view.add_representation('licorice',
                        color='blue',
                        selection='HSD LYS ARG')
view

NGLWidget()

Lets inspect the residues. A widget proves helpful here since there are many and we 
will need to be able to look up information about the atoms in each one.

In [50]:
@interact
def get_residue_atomInfo(Selected_Residue=[
    "%s_%g"%(iterTraj.top.residue(resID).name,
             iterTraj.top.residue(resID).index) \
    for resID in np.arange(iterTraj.top.n_residues)]):
    resNum=int(Selected_Residue.split('_')[1])+1
    atomDat=np.array(map(lambda x: x.replace('#','').split(),
        pt.resinfo(iterTraj.top,'atominfo :%g'%resNum,viewStruc[0],task='atominfo').split('\n')))
    return pd.DataFrame(data=list(atomDat[1:-1]),columns=atomDat[0])

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnU2VsZWN0ZWRfUmVzaWR1ZScsIG9wdGlvbnM9KHUnQVJHXzAnLCB1J0xFVV8xJywgdSdMRVVfMicsIHUnQVNQXzPigKY=


In [51]:
from pytraj.utils.progress import ProgressBarTrajectory

We now have the information we need to compute SASA data for each cationic residue (HSD, LYS, ARG).

We will only include atoms of the cationic regions.

This will yield a SASA calculation for each frame. The data table will be quite large.

We will process this raw data during the analysis step.

In [52]:
basicResidueIDs=[res.original_resid for res in \
                 viewStruc.top.residues if res.name in ['HSD','LYS','ARG']]

masksDict={
    'ARG':'@NE,NH1,NH2,HE,HH11,HH12,HH21,HH22,CZ',
    'LYS':'@NZ,HZ1,HZ2,HZ3',
    'HSD':'@CG,CE1,CE2,CD2,ND1,NE2,HD1,HD2',
}

commandList=['surf :%g%s'%(iterTraj.top.residue(resID-1).original_resid,
                           masksDict[iterTraj.top.residue(resID-1).name]) \
             for resID in basicResidueIDs]
pbar=ProgressBarTrajectory(iterTraj,style='tqdm')
SASA_data=pt.compute(commandList,pbar)

In [54]:
basicResidueSASAtable=pd.DataFrame(np.arange(iterTraj.n_frames),columns=['Frame'])
basicResidueSASAtable['System']='Piezo_PIP2'
for ii,resID in tqdm.tqdm_notebook(enumerate(basicResidueIDs)):
    entryName='%s_%g'%(iterTraj.top.residue(resID-1).name,resID)
    basicResidueSASAtable[entryName]=SASA_data.values()[ii]
basicResidueSASAtable=basicResidueSASAtable[np.concatenate([
    ['System','Frame'],
    basicResidueSASAtable.columns[2:]])]
basicResidueSASAtable.to_csv('basicResidueSASA.csv',index=False)
basicResidueSASAtable.head()

,System,Frame,ARG_1,ARG_15,ARG_21,ARG_22,HSD_27,LYS_30,LYS_42,ARG_63,...,HSD_4198,ARG_4210,LYS_4213,ARG_4222,ARG_4225,LYS_4236,ARG_4242,LYS_4249,ARG_4252,ARG_4254
0,Piezo_PIP2,0,87.971682,72.998160,80.786466,31.312725,67.773444,20.623376,55.209823,135.463408,...,43.275383,69.884503,24.943980,4.626739,88.281732,16.574375,15.564438,38.770691,96.299466,130.583524
1,Piezo_PIP2,1,76.917992,102.304355,79.381224,13.995005,86.148243,33.957993,55.538281,128.435914,...,77.561030,116.671583,39.405101,36.539325,56.898822,23.242080,33.811875,34.774065,88.537691,131.083016
2,Piezo_PIP2,2,92.678538,90.994392,68.379834,15.453099,90.343862,27.959345,53.981236,125.491502,...,52.222729,108.734830,46.284419,32.924167,102.064085,55.765358,36.739432,49.450528,125.012244,130.259988
3,Piezo_PIP2,3,88.609785,87.762004,72.502762,1.874848,84.170820,14.094714,57.859521,128.619427,...,57.925787,78.240005,41.645692,24.706268,97.861685,57.142842,24.409060,61.424450,105.862211,120.377420
4,Piezo_PIP2,4,104.101779,87.028277,98.756076,4.509094,88.035775,19.995091,51.474539,130.784876,...,57.793013,76.847820,39.216604,34.756950,88.014204,64.173321,45.711059,61.534140,101.307814,128.690753
